# Introduction

Welcome to the Eureka! tutorial - today we will learn how to run Eureka!'s S3 data reduction module, which takes 2D JWST data and reduces it to 1D spectra.

Eureka! is an open-source python package 

## Goals
- walk through all the major steps in data reduction
- get comfortable with the Eureka! structure and syntax
- most importantly, make sure none of the steps are a black box.

## Import standard python packages and Eureka!

In [43]:
import sys, os, time                                                            
import numpy as np                                                              
from importlib import reload                                                    
import eureka.S3_data_reduction.s3_reduce as s3                                 
from eureka.lib import readECF as rd                                            
from eureka.lib import logedit                                                  
from eureka.lib import readECF as rd                                            
from eureka.lib import manageevent as me                                        
from eureka.S3_data_reduction import optspex                                    
from eureka.lib import astropytable                                             
from eureka.lib import util                                                     
from eureka.S3_data_reduction import plots_s3

### Step 0: Initialization 

In [44]:
# Starts timer to monitor how long data reduction takes
t0      = time.time()

# Names the event (has to match the event name used for the *.pcf files)
eventlabel = 'wasp43b'                                                          
                                                                                
# Initialize metadata object to store all extra information 
# related to the event and the data reduction 

meta              = s3.Metadata()                                                 
meta.eventlabel   = eventlabel                                                    
                                                                                
# Initialize data object to store 2D data from the observation                                                       
dat              = s3.Data()   

Try printing out the time by typing ``print(t0)`` in a code cell. Run it again. Do you see the time change?

In [45]:
# Load Eureka! control file and store values in Metadata object                    
ecffile = 'S3_' + eventlabel + '.ecf'                                           
ecf     = rd.read_ecf(ecffile)                                                  
rd.store_ecf(meta, ecf)  

The ECF ("Eureka control file") is now stored in a Metadata object. This includes all the high level information about the data reduction (which JWST instrument was used? do we want to display plots? where is the data stored? what size is the extraction window? etc.)

To see the current contents of the Metadata object, type ``meta.__dict__.keys``. 

What is the value of ``meta.bg_deg``? Can you change it?

### Step 1: Make directories to store reduced data, create log file, read in data

In [46]:
# Create directories for Stage 3 processing                                     
datetime= time.strftime('%Y-%m-%d_%H-%M-%S')                                    
meta.workdir = 'S3_' + datetime + '_' + meta.eventlabel                             
if not os.path.exists(md.workdir):                                              
    os.makedirs(meta.workdir)                                                     
if not os.path.exists(meta.workdir+"/figs"):                                      
    os.makedirs(meta.workdir+"/figs")                                             
                                                                                
# Load instrument module                                                        
exec('from eureka.S3_data_reduction import ' + meta.inst + ' as inst', globals()) 
reload(inst)                                                                    
                                                                                
# Open new log file                                                             
meta.logname  = './'+meta.workdir + '/S3_' + meta.eventlabel + ".log"                 
log         = logedit.Logedit(meta.logname)                                       
log.writelog("\nStarting Stage 3 Reduction")                                    
                                                                                
# Create list of file segments                                                  
meta = util.readfiles(meta)                                                         
num_data_files = len(meta.segment_list)                                           
log.writelog(f'\nFound {num_data_files} data file(s) ending in {meta.suffix}.fits')
                                                                                
stdspec = np.array([])


Starting Stage 3 Reduction

Found 21 data file(s) ending in calints.fits


*Important check!* Were the correct files read in? They are stored in ``meta.segment_list``.

### Step 2: read the data!

In [47]:
# pick a single file to read and reduce as a test
m = 17

# Read in data frame and header                                             
log.writelog(f'Reading file {m+1} of {num_data_files}')                     
dat = inst.read(meta.segment_list[m], dat, returnHdr=True)

Reading file 18 of 21


#### What data are we using?

The full description of the data is available [here](https://stsci.app.box.com/s/8r6kqh9m53jkwkff0scmed6zx42g307e/file/804595804746)). To quickly summarize, we are using simulated NIRCam grism time series data from the ERS Simulated Spectra Team.  The simulation assumes a WASP-43 b-like planet with physically realistic spectral features added. The simulated data are based on the following observational design:

- GRISMR+F322W2 pupil and filter
- RAPID readout mode
- 19 Groups per integrations
- 1287 integrations
- 1 Exposure
- 4 Output amplifiers
The data themselves are divided into “segments,” with each individual segment (seg001, seg002, etc.) containing a subset of the overall dataset. This is how flight data will be delivered. The segments are numbered in their order of observation.

We will use the Stage 2 Output from the [JWST data reduction pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/stages.html). For NIRCam, Stage 2 consists of the flat field correction, WCS/wavelength solution, and photometric calibration (counts/sec -> MJy). Note that this is specifically for NIRCam: the steps in Stage 2 change a bit depending on the instrument. The Stage 2 outputs are rougly equivalent to a “flt” file from HST.

The files have the suffix ``/*calints.fits`` which contain fully calibrated images (MJy) for each individual integration. This is the one you want if you’re starting with Stage 2 and want to do your own spectral extraction.